In [1]:
'''
Sanjay Singh
san.singhsanjay@gmail.com
June-2021
To train a simple CNN on MNIST dataset
'''

'\nSanjay Singh\nsan.singhsanjay@gmail.com\nJune-2021\nTo train a simple CNN on MNIST dataset\n'

In [2]:
# packages
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [3]:
# constants
IMG_W = 28
IMG_H = 28
IMG_C = 3
LR = 0.01
BATCH_SIZE = 70
CLASSES = 10
EPOCHS = 5

In [4]:
# paths
image_data_path = "/notebooks/trainingSet/"
train_csv_path = "/notebooks/intermediate_outputs/train_data.csv"
val_csv_path = "/notebooks/intermediate_outputs/val_data.csv"
target_path = "/notebooks/trained_model/"

In [5]:
# function to generate data and pass in batches
def data_generator(train_df):
	while(True):
		image_batch = np.ndarray((BATCH_SIZE, IMG_W, IMG_H, IMG_C))
		image_label = np.ndarray((BATCH_SIZE, CLASSES))
		n = 0
		for i in range(train_df.shape[0]):
			dir_name = str(train_df.iloc[i]['label']) + "/"
			image_batch[n] = tf.keras.preprocessing.image.load_img(image_data_path + dir_name + train_df.iloc[i]['imagename'])
			image_label[n] = tf.one_hot(train_df.iloc[i]['label'], CLASSES)
			n = n + 1
			if(n == BATCH_SIZE):
				n = 0
				yield image_batch, image_label

In [6]:
# loading train_csv data
train_df = pd.read_csv(train_csv_path)
print("Shape of train_df: ", train_df.shape)

Shape of train_df:  (35000, 2)


In [7]:
# loading val_csv data
val_df = pd.read_csv(val_csv_path)
print("Shape of val_df: ", val_df.shape)

Shape of val_df:  (2000, 2)


In [8]:
# loading val data
val_images = np.ndarray((val_df.shape[0], IMG_W, IMG_H, IMG_C))
val_labels = np.ndarray((val_df.shape[0], CLASSES))
for i in range(val_df.shape[0]):
	dir_name = str(val_df.iloc[i]['label']) + "/"
	val_images[i] = tf.keras.preprocessing.image.load_img(image_data_path + dir_name + val_df.iloc[i]['imagename'])
	val_labels[i] = tf.one_hot(val_df.iloc[i]['label'], CLASSES)

In [9]:
# defining CNN architecture
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_W, IMG_H, IMG_C)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1982528   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

In [10]:
# compile model
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [11]:
# train model
steps = train_df.shape[0] // BATCH_SIZE
generator = data_generator(train_df)
history = model.fit_generator(generator, epochs=EPOCHS, steps_per_epoch=steps, validation_data=(val_images, val_labels), 
verbose=1)
model.save(target_path + "model_v1.h5")
print("Model saved successfully")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
500/500 [==============================] - 148s 282ms/step - loss: 2.7918 - accuracy: 0.8198 - val_loss: 0.0972 - val_accuracy: 0.9715
Epoch 2/5
500/500 [==============================] - 60s 120ms/step - loss: 0.0966 - accuracy: 0.9706 - val_loss: 0.0765 - val_accuracy: 0.9795
Epoch 3/5
500/500 [==============================] - 60s 120ms/step - loss: 0.0519 - accuracy: 0.9833 - val_loss: 0.1127 - val_accuracy: 0.9745
Epoch 4/5
500/500 [==============================] - 60s 119ms/step - loss: 0.0427 - accuracy: 0.9865 - val_loss: 0.0875 - val_accuracy: 0.9770
Epoch 5/5
500/500 [==============================] - 59s 119ms/step - loss: 0.0390 - accuracy: 0.9866 - val_loss: 0.0897 - val_accuracy: 0.9825


In [16]:
file_pi = open(target_path + "history_v1", 'wb')
pickle.dump(history.history, file_pi)
file_pi.close()
print("Successfully saved history of model")

Successfully saved history of model
